# Practical session: Interpretability in Machine learning

Machine learning algorithms often act as black boxes, with their decision-making processes being opaque. In this practical session, we aim to shed light on these processes, enhancing our understanding and interpretability of machine learning models. We will start by exploring model agnostic methods, which are techniques applicable to any machine learning model, regardless of its internal workings. Following this, we will delve into specific techniques tailored for neural network models.

## Model agnostic Methods

To begin, we will tackle a simple regression problem. This hands-on exercise will serve as an introduction to the tools and techniques used for model interpretation.

### Dataset

We will begin with a simple regression problem of predicting Californian houses' house prices according to 8 numerical features.  
Scikit-learn provides the dataset, and a full description is available [here](https://scikit-learn.org/stable/datasets/real_world.html#california-housing-dataset).


In [ ]:
import pandas as pd
import numpy as np
from sklearn.datasets import fetch_california_housing

cal_housing = fetch_california_housing()
feature_names = cal_housing.feature_names
X = pd.DataFrame(cal_housing.data, columns=feature_names)
y = cal_housing.target
X.head()

I encourage you to explore the dataset by yourself using pandas and seaborn; it is always a good exercise.  
Nonetheless, today's practical session is not designed to train the best possible models but to learn how to interpret them.  
It may be a good opportunity to present you a friendly tool for exploring datasets: [YData profiling](https://docs.profiling.ydata.ai/latest/).  
YData profiling provides an automatic data exploration tool and html reports in a one-liner.  
I would recommend usually doing it by yourself in other projects, but it may still be helpful for having a quick overview of what your dataset looks like.

In [ ]:
!pip install ydata-profiling > /dev/null 2>&1

In [ ]:
from ydata_profiling import ProfileReport

ProfileReport(X, sort=None)

Now use scikit-learn's ```train_test_split``` method to split your dataset in train and test (10%)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = ...

Train a linear regression, a random forest and a neural network to predict the houses price.  
Test all models on your test set to copare their performances.  
Feel free to train and test more sophisticated models such as XGBoost, LightGBM...

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import QuantileTransformer, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor

lr = ...

rf = ...


mlp = ...

lr.fit(X_train, y_train)
rf.fit(X_train, y_train)
mlp.fit(X_train, y_train)


 ... #test your models

Linear models are considered intrinsically interpretable.  
Using the ```coef_``` attribute of your model, visualize the importance of each of the features of the linear model.  

In [ ]:
# to access the model part of your pipeline: lr[1]

...

### Features importance

While Scikit-learn offers native methods for computing feature importance in tree-based models, we will explore a more versatile approach suitable for various model types.

We will first utilize the feature permutation method, implemented in an external library called [Eli5](https://eli5.readthedocs.io/en/latest/overview.html#features).  
Feature permutation is a model-agnostic technique that measures the importance of a feature by calculating the increase in the model's prediction error after permuting the feature's values. This method disrupts the relationship between the feature and the target, thereby highlighting the feature's influence on the model's predictive accuracy. By using feature permutation, we can evaluate the impact of each feature across our three different models.



In [ ]:
!pip install eli5 > /dev/null 2>&1

Use the ```PermutationImportance``` to compute the features importance of your models.  (Documentation [here](https://eli5.readthedocs.io/en/latest/blackbox/permutation_importance.html)).  
Plot them for each of your model.  
Are the feature importance of the linear model similar to the coefficients?
Are the features as important for all your models?  
Create a dictionnary containing the top 5 features for each of your model (**key**:model name, **value**: dataframe of features importance)

In [ ]:
import eli5
from eli5.sklearn import PermutationImportance
import matplotlib.pyplot as plt
import seaborn as sns

features_importance_dict = {}
for model, name in zip([lr, rf, mlp], ['logistic regression', 'random forest', 'multi layer perceptron']):
  plt.figure()
  permumtation_import = PermutationImportance(...)
  features_importance = {'Feature_name':feature_names, 'Importance':permumtation_import.feature_importances_}  
  features_importance = pd.DataFrame(features_importance) # dataframe containing the features names and their importance
  features_importance = features_importance.sort_values(...) # sort the dataframe by feature importance
  features_importance_dict[name] = ... #add the dataframe to your dictionnary
  ax = sns.barplot(x=..., y=..., data=features_importance) #plot the model's features importance
  plt.title(name)

If you've noticed that geographical location is an important feature in some of your models, you can use a tool called [folium](http://python-visualization.github.io/folium/) to visualize this. Folium makes it easy to plot these features on a map, giving us a clearer picture of how they affect house prices.

In [ ]:
import folium

latmean = X.Latitude.mean()
lonmean = X.Longitude.mean()
map = folium.Map(location=[latmean,lonmean],
        zoom_start=6)

def color(value):
    if value in range(0,149999):
        col = 'green'
    elif value in range(150000,249999):
        col = 'yellow'
    elif value in range(250000,349999):
        col = 'orange'
    else:
        col='red'
    return col

map = folium.Map(location=[latmean,lonmean],
        zoom_start=6)

# Top three smart phone companies by market share in 2016
for lat,lan,value in zip(X_test['Latitude'][:300],X_test['Longitude'][:300],y_test[:100]*100000):
    folium.Marker(location=[lat,lan],icon= folium.Icon(color=color(value),icon_color='black',icon = 'home')).add_to(map)
map

### PDP and ICE Plots

Partial Dependence Plots (PDP) and Individual Conditional Expectation (ICE) plots are powerful tools for interpreting machine learning models. They help us understand the relationship between the target response and a set of input features of interest.

For generating these plots, we will use the [pdpbox](https://pdpbox.readthedocs.io/en/latest/) library. PDP shows the average effect of a feature on the prediction, while ICE plots display this effect for individual instances, providing a more detailed view.


In [ ]:
!pip install pdpbox > /dev/null 2>&1

The following code shows you how to produce a PDP plot for the random forest model.  
```python
from pdpbox import pdp, get_dataset, info_plots

pdp_feat = pdp.PDPIsolate(model=rf, dataset=X_test.copy(), # need to copy() since PDPIsolate modifies the df
                            model_features=feature_names, feature='MedInc',
                            n_classes=0, num_grid_points=50)
pdp_feat.plot(engine='matplotlib', plot_lines=True)
```
Use it to generate the PDP plots for the three most important features of each of your models.  
What is the nature of their relationship with the target?


In [ ]:
from pdpbox import pdp
model = rf #lr, mlp
model_name = 'random forest'#'logistic regression' , 'multi layer perceptron'

top_3_features = features_importance_dict[model_name].Feature_name[:3].values
for i, feature in enumerate(top_3_features, 1):
  pdp_dist = ...
  ...

It is also possible to visualize the combined effetc of two features:

In [ ]:
interact = pdp.PDPInteract(model=model, df=X_test.copy(), model_features=feature_names,
                             features=['Latitude', 'Longitude'],
                             feature_names=['Latitude', 'Longitude'],
                             n_classes=0)
interact.plot(engine='matplotlib', plot_type='contour')

Scikit-learns also provides methods to generate such plots, but may offer less flexibility.

In [ ]:
from sklearn.inspection import partial_dependence
from sklearn.inspection import PartialDependenceDisplay

for model, model_name in zip([lr, rf, mlp], ['logistic regression', 'random forest', 'multi layer perceptron']):

  top_3_features = features_importance_dict[name].Feature_name[:3].values
  display = PartialDependenceDisplay.from_estimator(
    model,
    X_test,
    top_3_features,
    kind="both",
    subsample=50,
    n_jobs=3,
    n_cols=3,
    grid_resolution=20,
    random_state=0,
    ice_lines_kw={"color": "tab:blue", "alpha": 0.2, "linewidth": 0.5},
    pd_line_kw={"color": "tab:orange", "linestyle": "--"}
    )
  display.figure_.suptitle(f"Partial dependence for {model_name} model")
  display.figure_.subplots_adjust(hspace=0.3)

In [ ]:
for model, name in zip([lr, rf, mlp], ['logistic regression', 'random forest', 'multi layer perceptron']):
  _, ax = plt.subplots(ncols=3, figsize=(9, 4))
  top_2_features = features_importance_dict[name].Feature_name[:3].values
  features = [top_2_features[0], top_2_features[1], (top_2_features[0], top_2_features[1])]
  display = PartialDependenceDisplay.from_estimator(
      model,
      X_test,
      features,
      kind="average",
      n_jobs=3,
      grid_resolution=20,
      ax=ax,
  )
  display.figure_.suptitle(f"Partial dependence for {name} model")
  display.figure_.subplots_adjust(wspace=0.4, hspace=0.3)

### SHAP

Previous methods provided global explanations of our models.  
We will now focus on local interpretability methods.  
We will begin with the SHAP methods based on the estimation of the Shapley values.  
The library [SHAP](https://github.com/shap/shap) implements the SHAP method (and many others).

Inspire yourself with the following [examples](https://github.com/shap/shap) to produce a visualization of the estimated Shapley values of your different models, first for a single example using the ```force``` method and for the entire test, dataset using the ```summary_plot``` method.

In [ ]:
!pip install shap > /dev/null 2>&1

In [ ]:
%matplotlib inline
import matplotlib
import shap
shap.initjs() #needed to plot results directly on the notebook

idx = 1 # index of the instance we want to explain

explainer = ...
shap_values = explainer(X_test.iloc[0:20,:]) #To speed up we just compute the shap values for 100 exemples
... # use option , matplotlib=matplotlib to display in colab

In [ ]:
shap.summary_plot(shap_values, X_test.iloc[0:20,:])

### LIME (Local Interpretable Model-agnostic Explanations)

LIME is an innovative technique for explaining the predictions of any machine learning classifier in an understandable way. It helps in understanding why a model makes certain predictions, focusing on individual instances.

During our class, we discussed LIME as a model agnostic local interpretability method. This approach is valuable for gaining insights into specific predictions, especially in complex models. For practical application, we will use the [LIME implementation provided by the original authors](https://github.com/marcotcr/lime), which is well-documented and widely used in the Python community.


In [ ]:
!pip install lime > /dev/null 2>&1

LIME provides eay to understand an friendly looking explanations for your model predictions.  
You first need to instanciate an Explainer (in our case a ```LimeTabularExplainer```) and then call the ```explain instance``` method of the explainer to get the explanations.  

In [ ]:
import lime
import lime.lime_tabular

index = 0

explainer = lime.lime_tabular.LimeTabularExplainer(X_test.values, feature_names=feature_names, mode="regression")
exp = explainer.explain_instance(X_test.iloc[index], rf.predict, num_features=5, top_labels=1)
exp.show_in_notebook(show_table=True, show_all=True)

#### Classification
LIME also works with classification problems.  
We will repeat the previous experiment using a different dataset for [breast cancer prediction](https://scikit-learn.org/stable/datasets/toy_dataset.html#breast-cancer-dataset) and a decision trees algorithm.

In [ ]:
from sklearn.datasets import load_breast_cancer

breast_cancer = load_breast_cancer()
feature_names = breast_cancer.feature_names
target_names = breast_cancer.target_names
X = pd.DataFrame(breast_cancer.data, columns=feature_names)
y = breast_cancer.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=0)
X_train.head()

Train a decision tree (with max_depth=5) on this dataset and plot the confusion matrix on the test dataset.

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import ConfusionMatrixDisplay

dt = ...
...
print(f"Descision Tree score: {dt.score(X_test, y_test):.2f}")

...

Decision trees are also interpretable models.  
Scikit-learn provides an efficient way to visualize their structure.  

In [ ]:
import matplotlib.pyplot as plt
import sklearn.tree as tree
plt.figure(figsize=(20,20))
tree.plot_tree(dt[1])
plt.show()

Explain the predictions of your model on some examples.  
For classification tasks, LIME needs the predicted "probailities" of the model.  
Use the ```predict_proba``` method of your classifier instead of the ```predict``` method when calling the explain instance.    
Also, don't forget to remove the ```mode="regression"``` argument when instanciating the ```LimeTabularExplainer```.  
Are the explanations consistent with the decision graph?

In [ ]:
explainer = lime.lime_tabular.LimeTabularExplainer(...)

In [ ]:
index = 0
exp = explainer.explain_instance(...)
exp.show_in_notebook(show_table=True, show_all=True)

#### Text data
The authors also provided nice visualizations for text data.  
We will now train a Random forest to classify whether scientific texts are about medicine or space.   
We will be using two categories from the [newsgroup dataset](https://scikit-learn.org/0.19/datasets/twenty_newsgroups.html) available in scikit-learn.

In [ ]:
from sklearn.datasets import fetch_20newsgroups

categories = [
    'sci.med',
    'sci.space'
]


train_data = fetch_20newsgroups(subset='train', categories=categories)
test_data = fetch_20newsgroups(subset='test', categories=categories)

class_names = train_data.target_names

X_train, y_train  = train_data.data, train_data.target
X_test, y_test = test_data.data, test_data.target

Here is an example of text to classify:

In [ ]:
X_train[0]

Let's train a random forest to classify our dataset.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import make_pipeline

model = make_pipeline(
        CountVectorizer(max_df= 0.5, ngram_range= (1, 2)),
        TfidfTransformer(),
        RandomForestClassifier()
)

model.fit(X_train, y_train)
print(f"Model score: {model.score(X_test, y_test):.2f}")

We will use a specific ```LimeTextExplainer``` to explain the predictions.

In [ ]:
from lime.lime_text import LimeTextExplainer
explainer = LimeTextExplainer(class_names=class_names)

In [ ]:
from lime.lime_text import LimeTextExplainer
explainer = LimeTextExplainer(class_names=class_names)

index = 11
exp = explainer.explain_instance(X_test[index], model.predict_proba, num_features=6)

prediction = model.predict_proba([X_test[index]])
class_predicted = class_names[prediction.argmax(1)[0]]
class_proba = prediction.max(1)[0]
true_class = class_names[y_test[index]]
print(f'Class predicted: {class_predicted} (p={class_proba})')
print(f'True class: {class_names[y_test[index]]}')

Here are the top words used by the classifier.  

In [ ]:
fig = exp.as_pyplot_figure()

These explanations seem plausible, let's visualize these words in their context:

In [ ]:
exp.show_in_notebook(text=True)

Some of the words are in the newsgroup header!  
Would you trust such a classifier?

Scikit-learn provides an option to remove all headers and footers.
Train a new model on the datset with removed headers and footers and comare its F1-score with the previous model.

In [ ]:
train_data = fetch_20newsgroups(subset='train',remove=('headers', 'footers', 'quotes'),
                                     categories=categories)


class_names = train_data.target_names

X_train, y_train  = train_data.data, train_data.target
X_test, y_test = test_data.data, test_data.target


model = ...
...
model.fit(X_train, y_train)
print(f"Model score: {model.score(X_test, y_test):.2f}")

Now visualize the explainations computed by lime on the same example with your new model.  
Which of the two models would you trust the more?

In [ ]:
explainer = ...
exp = ...

prediction = ...
class_predicted = ...
class_proba = ...
true_class = ...
...

#### Image Data

Finally, LIME also provides friendly-looking visualizations on images.  
We will use a subset of Imagenet to test these visualizations.


In [ ]:
!wget https://s3.amazonaws.com/fast-ai-imageclas/imagenette2.tgz > /dev/null 2>&1
!tar zxvf imagenette2.tgz > /dev/null 2>&1

In [ ]:
import torchvision
import torchvision.transforms as transforms
import torch
import os
from torch.utils.data import Dataset

means, stds = (0.485, 0.456, 0.406), (0.229, 0.224, 0.225)
train_transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(means, stds),
    ])

test_transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(means, stds),
    ])

def get_imagenette2_loaders(root_path='./imagenette2', **kwargs):

    trainset = torchvision.datasets.ImageFolder(os.path.join(root_path, "train"), transform=train_transform)
    trainloader = torch.utils.data.DataLoader(trainset, **kwargs)
    testset = torchvision.datasets.ImageFolder(os.path.join(root_path, "val"), transform=test_transform)
    testloader = torch.utils.data.DataLoader(testset, **kwargs)
    return trainloader, testloader

trainloader, testloader = get_imagenette2_loaders( batch_size=64, shuffle=True, num_workers=2)

labels = ['tench', 'English springer', 'cassette player', 'chain saw', 'church', 'French horn', 'garbage truck', 'gas pump', 'golf ball', 'parachute']

In [ ]:
from torchvision.utils import make_grid
import matplotlib.pyplot as plt

inv_normalize = transforms.Normalize(
   mean= [-m/s for m, s in zip(means, stds)],
   std= [1/s for s in stds]
)

x, _ = next(iter(trainloader))
img_grid = make_grid(x[:16])
img_grid = inv_normalize(img_grid)
plt.figure(figsize=(20,15))
plt.imshow(img_grid.permute(1, 2, 0))
plt.axis('off')

We will train a neural network to classify these images.  
However, training neural networks on high-definition images may be long and difficult.  
We will use a pre-trained VGG11 and replace the fully connected part of the network to match the ten classes (this is called transfer learning).  
Complete the following code to instantiate a model predicting among ten classes with pre-trained features.

In [ ]:
import torch.nn as nn
import torch

model = torchvision.models.vgg11(pretrained=True)
for param in model.features:
    param.requires_grad = False

model.classifier = nn.Sequential(
            ...
        )

model = model.cuda()

Fill the following code to implement the training loop.

In [ ]:
from tqdm import tqdm

criterion_classifier = nn.CrossEntropyLoss(reduction='mean')

def train(model, optimizer, trainloader, epochs=30):
    t = tqdm(range(epochs))
    for epoch in t:
        corrects = 0
        total = 0
        for x, y in trainloader:
            loss = 0
            x = x.cuda()
            y = y.cuda()
            y_hat = ...

            loss += criterion_classifier(...)
            _, predicted = y_hat.max(1)
            corrects += predicted.eq(y).sum().item()
            total += y.size(0)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            t.set_description(f'epoch:{epoch} current accuracy:{round(corrects / total * 100, 2)}%')
    return (corrects / total)

Train your model. One or two epochs should be enough since we are using transfer learning.

In [ ]:
learning_rate = 5e-3
epochs = 1
optimizer = torch.optim.Adam(model.classifier.parameters(), lr=learning_rate)
...

Test your network to validate it has enough classification abilities.

In [ ]:
def test(model, dataloader):
    test_corrects = 0
    total = 0
    with torch.no_grad():
        for x, y in dataloader:
            x = x.cuda()
            y = y.cuda()
            y_hat = model(x).argmax(1)
            test_corrects += y_hat.eq(y).sum().item()
            total += y.size(0)
    return test_corrects / total

model.eval()
test_acc = test(model, testloader)
print(f'Test accuracy: {test_acc:.2f} %')

Using a single example, we will now use lime to visualize the important parts of the image for our model prediction.

In [ ]:
idx = 0

img = inv_normalize(x[idx])
np_img = np.transpose(img.cpu().detach().numpy(), (1,2,0))*255
np_img = np_img.astype(np.uint8)
plt.imshow(np_img)
plt.axis('off')

Let's first verify our model prediction:

In [ ]:
input = x[idx].unsqueeze(0).cuda()
output = model(input)
_, prediction = torch.topk(output, 1)
print(f"Model's prediction: {labels[prediction.item()]}")

LIME provide a ```LimeImageExplainer``` to deal with images.
However, the  ```LimeImageExplainer``` requires a callable function that will directly produce predictions for a list of images (the perturbed from the original images) in the form of a numpy array.  
Our pytorch model works with pytorch ```Tensor``` mini-batches and outputs ```Tensor``` objects.  
We thus need to wrap our model and the associated pre-processing into a single callable function to use the ```LimeImageExplainer```.

In [ ]:
import torch.nn.functional as F

lime_transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(means, stds),
    ])

def batch_predict(images):
  with torch.no_grad():
    model.eval()
    batch = torch.stack(tuple(lime_transform(i) for i in images), dim=0)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    batch = batch.to(device)
    logits = model(batch)
  return logits.detach().cpu().numpy()

We now can use the Lime explainer to visualize which parts of the images were the most important.

In [ ]:
from lime import lime_image
from skimage.segmentation import mark_boundaries

explainer = lime_image.LimeImageExplainer()
explanation = explainer.explain_instance(np_img,
                                         batch_predict, # classification function
                                         top_labels=5,
                                         hide_color=0,
                                         num_samples=1000)

temp, mask = explanation.get_image_and_mask(explanation.top_labels[0], positive_only=False, num_features=10, hide_rest=False)
img_boundry = mark_boundaries(temp/255.0, mask)
plt.imshow(img_boundry)

## Model specific methods
We just saw many methods for model agnostic interpretability.  
The second part of this session is dedicated to model-specific methods.  
In particular, we will implement methods specific to neural networks.

### Vanilla gradient back-propagation

We will now implement three methods to generate saliency maps on our images.  
One of the simplest ways to generate saliency maps is certainly to backpropagate the gradients of the predicted output directly to the image input.  This method, called vanilla gradient backpropagation, is presented in this [article](Deep Inside Convolutional Networks: Visualising Image Classification Models and Saliency Maps).

Let's first visualize an image for which we will generate a saliency map according to our model's prediction.

In [ ]:
import numpy as np

x, _ = next(iter(trainloader))
idx = 0

img = x[idx]
np_img = np.transpose(inv_normalize(img).cpu().detach().numpy(), (1,2,0))
plt.imshow(np_img)
plt.axis('off')

By default, input tensors do not require to generate gradients in Pytorch.  
Thus, we first need to set the image to catch the gradient during the backward pass.

In [ ]:
img = img.unsqueeze(0).cuda() # we need to set the input on GPU before the requires_grad operation!
img.requires_grad_();

We will now compute the model's prediction for this image and backpropagate from this prediction to the image.

In [ ]:
output = model(img)
output_idx = output.argmax()
output_max = output[0, output_idx]

output_max.backward()

We can now generate a saliency map were important pixels will correspond to important gradients!

In [ ]:
saliency, _ = torch.max(img.grad.data.abs(), dim=1)
saliency = saliency.squeeze(0)


plt.figure(figsize=(15,10))
plt.subplot(1,2,1)
plt.imshow(np_img)
plt.axis('off')
plt.subplot(1,2,2)
plt.imshow(saliency.cpu(), cmap='hot')
plt.axis('off')


Try with different images.

In [ ]:
...

### Smooth grad
A simple way to generate smoother visualizations called [Smooth-grad](https://arxiv.org/pdf/1706.03825.pdf) consists of averaging saliency maps from augmented versions of the original image.  
Complete the following function to generate the gradient of an image according to the model's prediction.

In [ ]:
def get_vanilla_grad(img, model):
  ...#retain gradients
  ...

  return img.grad

We will now generate perturbated versions of the image by adding a gaussian noise to the original image.  
For every image generated we will compute the corresponding gardients and average them to generate the final saliency map.

In [ ]:
import numpy as np

stdev_spread=0.15
n_samples=100
stdev = stdev_spread * (img.max() - img.min())
total_gradients = torch.zeros_like(img, device='cuda')

for i in range(n_samples):
    noise = np.random.normal(0, stdev.item(), img.shape).astype(np.float32)
    noisy_img = img + torch.tensor(noise, device='cuda', requires_grad=True)
    grad= get_vanilla_grad(noisy_img, model)
    total_gradients += grad * grad #using the square of the gradients generates smoother visualizations
    #total_gradients += grad
total_gradients /= n_samples

saliency, _ = torch.max(total_gradients.abs(), dim=1)
saliency = saliency.squeeze(0)


plt.figure(figsize=(15,10))
plt.subplot(1,2,1)
plt.imshow(np_img)
plt.axis('off')
plt.subplot(1,2,2)
plt.imshow(saliency.cpu(), cmap='hot')
plt.axis('off')

Try it with other images.

In [ ]:
...

### Grad-CAM
Instead of propagating the gradients to the inputs, the [Grad-CAM](https://arxiv.org/abs/1610.02391) method generates a saliency map by multiplying the outputs of the final feature map with an average of its gradients.  It thus generates coarse saliency maps, sometimes more relevant than pixels.  
We will create a 'hook' to keep both the activations and the gardients of a network layer.  
This operation is a bit tricky, just keep it mind that it is a way to keep activations and gradients in a single object during the forward and the backward pass.

In [ ]:
class HookFeatures():
    def __init__(self, module):
        self.feature_hook = module.register_forward_hook(self.feature_hook_fn)
    def feature_hook_fn(self, module, input, output):
        self.features = output.clone().detach()
        self.gradient_hook = output.register_hook(self.gradient_hook_fn)
    def gradient_hook_fn(self, grad):
        self.gradients = grad
    def close(self):
        self.feature_hook.remove()
        self.gradient_hook.remove()

We will 'hook' the activations and gradients of the last convolutional layer.

In [ ]:
print(model)
hook = HookFeatures(model.features[19])

Similar to what we did before, we will backpropagate the gradients of the predicted output on the feature map this time and get both the activations and the gradients thanks to our hook on the last convolutional layer.

In [ ]:
output = model(img)
output_idx = output.argmax()
output_max = output[0, output_idx]
output_max.backward()

gradients = hook.gradients
activations = hook.features
pooled_gradients = torch.mean(gradients, dim=[0, 2, 3]) # we take the average gradient of every chanels
for i in range(activations.shape[1]):
    activations[:, i, :, :] *= pooled_gradients[i] # we multiply every chanels of the feature map with their corresponding averaged gradients

We can now take the average of all channels of the gradient weighted feature map to generate a heat map, keeping only the positive values to get the positive influences only.  
We also need to reshape the generated heat map to math the original input size.

In [ ]:
import cv2

heatmap = torch.mean(activations, dim=1).squeeze()
heatmap = np.maximum(heatmap.detach().cpu(), 0)
heatmap /= torch.max(heatmap)
heatmap = cv2.resize(np.float32(heatmap), (img.shape[2], img.shape[3]))
heatmap = np.uint8(255 * heatmap)
heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_RAINBOW) / 255
superposed_img = (heatmap) * 0.4 + np_img
plt.figure(figsize=(8,8))
plt.imshow(np.clip(superposed_img,0,1))
plt.axis('off')

We must remove our hook, it won't be used in the rest of the session.

In [ ]:
hook.close()

### Captum  

[Captum](https://captum.ai/)  is a library developed by Facebook to generate explanations on Pytorch models.  
It implements various other saliency maps methods that we did not cover during our class.
You should look at the doc and find out [other possible methods](https://captum.ai/docs/algorithms) to gain interpretability in your Pytorch models.  
We will quickly try some of them so you know how to use them if you need to.

In [ ]:
!pip install captum

In [ ]:
from captum.attr import IntegratedGradients
from captum.attr import GradientShap
from captum.attr import Occlusion
from captum.attr import NoiseTunnel
from captum.attr import GuidedGradCam
from captum.attr import DeepLift
from captum.attr import visualization as viz

In [ ]:
def plot_heatmap(attributions, img):
  _ = viz.visualize_image_attr_multiple(np.transpose(attributions.squeeze().cpu().detach().numpy(), (1,2,0)),
                              np.transpose(inv_normalize(img).squeeze().cpu().detach().numpy(), (1,2,0)),
                              methods=["original_image", "heat_map"],
                              signs=['all', 'positive'],
                              cmap='hot',
                              show_colorbar=True)

# Integradted gradients (https://arxiv.org/abs/1703.01365)
integrated_gradients = IntegratedGradients(model)
attributions = integrated_gradients.attribute(img, target=output_idx, n_steps=200, internal_batch_size=1)

plot_heatmap(attributions, img)

In [ ]:
#Noise tunnel (SmoothGrad, VarGrad: https://arxiv.org/abs/1810.03307)
noise_tunnel = NoiseTunnel(integrated_gradients)
attributions = noise_tunnel.attribute(img, nt_samples_batch_size=1, nt_samples=10, nt_type='smoothgrad_sq', target=output_idx)

plot_heatmap(attributions, img)

In [ ]:
#Occlusion (https://arxiv.org/abs/1311.2901)
occlusion = Occlusion(model)
attributions = occlusion.attribute(img,
                                  strides = (3, 8, 8),
                                  target=output_idx,
                                  sliding_window_shapes=(3,15, 15),
                                  baselines=0)

plot_heatmap(attributions, img)

In [ ]:
#DeepLift (https://arxiv.org/pdf/1704.02685.pdf)
dl = DeepLift(model)
attributions = dl.attribute(img, target=output_idx, baselines=img * 0)

plot_heatmap(attributions, img)

In [ ]:
#Guided Grad-CAM (https://arxiv.org/abs/1610.02391)
guided_gc = GuidedGradCam(model, model.features[19])
attribution = guided_gc.attribute(img, target=output_idx)
plot_heatmap(attributions, img)

## Feature visualization
Previous methods, generating saliency maps were model-specific local-interpretability methods.  
We will now implement a global interpretability called feature visualization.  
I really encourage you to read the [distill publication](https://distill.pub/2017/feature-visualization/) presenting the methods.  The obtained visualizations are superb!
The principle of the method is straightforward. It consists of optimizing a random image to maximize the output of a neural network unit.  
A unit can be a neuron, a channel of a feature map, or an entire feature map.  
In this practical session, we will focus on channels of a pre-trained VGG19 network.
Let's first have a look at our network architecture.

In [ ]:
model = torchvision.models.vgg19(pretrained=True).cuda()
model

Once again, we will create a hook to keep the activation of intermediate layers.  
These activations will be used to generate the feature visualizations and serve as a loss function to optimize the random image.  
We won't need to keep the gradients this time.

In [ ]:
class FeaturesHook():
    def __init__(self, module):
        self.hook = module.register_forward_hook(self.hook_fn)
    def hook_fn(self, module, input, output):
        self.features = output
    def close(self):
        self.hook.remove()

The following code will compute the feature visualization for one channel of one layer.  
We begin by initializing a random image and creating a hook on the desired layer.  
Then we will do a forward pass of our image through the network and compute a loss to maximize the hooked layer's desired channel.  
We will repeat this operation several epochs.

In [ ]:
def visualize_feature(model, layer_idx, channel_idx):
  img = torch.rand((1,3,224,224), requires_grad=True, device="cuda") #initialize a random image
  optimizer = torch.optim.Adam([img], lr=0.1, weight_decay=1e-6)
  features_hook =  FeaturesHook(model.features[layer_idx]) # hook the desired layer
  for n in range(512):
        optimizer.zero_grad()
        model(img) #forward pass
        features_map = features_hook.features
        # We normalize the features before computing the loss
        normalized_features = features_map / (torch.sqrt(torch.mean(torch.square(features_map))) + 1e-5)

        # Goal is to maximize the normalized channel's output
        loss = -normalized_features[0, channel_idx].mean()
        loss.backward()
        optimizer.step()

  features_hook.close()
  img = img.squeeze(0)
  img = inv_normalize(img).cpu().detach().numpy()
  img = np.transpose(img, (1,2,0))
  img = np.clip(img, 0, 1)
  plt.imshow(img)
  plt.axis('off')
  plt.tight_layout()

#visualize layer one channel 2
visualize_feature(model, 1, 3)

Complete the following code to generate feature visaulizations of various filters of layer 1.

In [ ]:
layer = 1

plt.figure(figsize=(20,25))
for i, filter in enumerate(range(0, 3), 1):
  ...

Now let's try with layer 5, and 35 for example.

In [ ]:
layer = 5

plt.figure(figsize=(20,25))
for i, filter in enumerate(range(0, 3), 1):
  ...

In [ ]:
layer = 35

plt.figure(figsize=(20,25))
for i, filter in enumerate(range(0, 3), 1):
  ...

Obtaining nice visualizations as in the original publication requires some additional tricks.  
For example, we can use a total variation loss to regularize the generated image.  
Total variation loss is a regularization technique that encourages smoothness in our generated image by penalizing rapid changes in pixel intensity.  
We can also use a Gaussian blur and Jitter to smooth the generated image.  
Finally we can also clip the pixel values to the [0,1] range to keep the image in the same range as the natural images.  
In fact, in the original publication, the authors used a more sophisticated regularization technique by optimizing the image in the Fourier space that we won't cover today.
Feel free to read the publication and try it on other more sophisticated networks.

In [ ]:
import random
import torch
import numpy as np
import matplotlib.pyplot as plt
from torchvision.transforms.functional import gaussian_blur

def jitter(img, max_jitter=1):
    jh, jw = random.randint(-max_jitter, max_jitter), random.randint(-max_jitter, max_jitter)
    return torch.roll(img, shifts=(jh, jw), dims=(2, 3))

def total_variation_loss(img):
    tv_h = torch.mean(torch.abs(img[:, :, 1:, :] - img[:, :, :-1, :]))
    tv_w = torch.mean(torch.abs(img[:, :, :, 1:] - img[:, :, :, :-1]))
    return tv_h + tv_w

def apply_gaussian_blur(img, kernel_size=3):
    return gaussian_blur(img, (kernel_size, kernel_size))

def visualize_feature(model, layer_idx, channel_idx, alpha=0.2, iterations=2048):
    img = torch.rand((1, 3, 224, 224), requires_grad=True, device="cuda")  # initialize a random image
    optimizer = torch.optim.Adam([img], lr=0.05)
    features_hook = FeaturesHook(model.features[layer_idx])  # hook the desired layer

    for n in range(iterations):
        optimizer.zero_grad()

        # Apply jitter
        img_jittered = jitter(img)

        # Apply Gaussian blur
        img_blurred = apply_gaussian_blur(img_jittered)

        model(img_blurred)  # forward pass with blurred image
        features_map = features_hook.features

        normalized_features = features_map / (torch.sqrt(torch.mean(torch.square(features_map))) + 1e-5)
        feature_loss = -normalized_features[0, channel_idx].mean()

        tv_loss = total_variation_loss(img)

        # Combine feature loss and total variation loss
        total_loss = feature_loss + alpha * tv_loss

        total_loss.backward()
        optimizer.step()

        # Pixel clipping
        with torch.no_grad():
            img.clamp_(-1, 1)

    features_hook.close()
    img = img.squeeze(0)
    img = inv_normalize(img).cpu().detach().numpy()
    img = np.transpose(img, (1, 2, 0))
    img = np.clip(img, 0, 1)
    plt.imshow(img)
    plt.axis('off')
    plt.tight_layout()

visualize_feature(model, 35, 12)


Use this code to generate feature visualizations of various filters of layer 35.  

In [ ]:
layer = 5

plt.figure(figsize=(20,25))
for i, filter in enumerate(range(0, 3), 1):
  ...

In [ ]:
layer = 35

plt.figure(figsize=(20,25))
for i, filter in enumerate(range(0, 3), 1):
  ...

We can also use this technique to generate images that maximize the activation of an output unit corresponding to a specific class.   
Here is an example for the ostrich class.  
Do you recognize some patterns?  
Try it on other classes. (list of classes [here](https://gist.github.com/yrevar/942d3a0ac09ec9e5eb3a))

In [ ]:
import random

def visualize_output_neuron(model, output_neuron_idx, alpha=0.25, iterations=2048):
    img = torch.rand((1, 3, 224, 224), requires_grad=True, device="cuda")  # initialize a random image
    optimizer = torch.optim.Adam([img], lr=0.1, weight_decay=1e-6)

    for n in range(iterations):
        optimizer.zero_grad()

        img_jittered = jitter(img)

        # Apply Gaussian blur
        img_blurred = apply_gaussian_blur(img_jittered)

        output = model(img_blurred)  # forward pass with blurred image

        # Maximize the activation of the chosen output neuron
        neuron_activation = output[0, output_neuron_idx]
        feature_loss = -neuron_activation

        # Compute the total variation loss
        tv_loss = total_variation_loss(img)

        # Combine feature loss and total variation loss
        total_loss = feature_loss + alpha * tv_loss

        total_loss.backward()
        optimizer.step()

        # Pixel clipping
        with torch.no_grad():
            img.clamp_(-1, 1)

    img = img.squeeze(0)
    img = inv_normalize(img).cpu().detach().numpy()
    img = np.transpose(img, (1, 2, 0))
    img = np.clip(img, 0, 1)
    plt.imshow(img)
    plt.axis('off')
    plt.tight_layout()

visualize_output_neuron(model, 9) # 'ostrich, Struthio camelus'

In [ ]:
visualize_output_neuron(model, 1, alpha=0.25) # jellyfish

In [ ]:
visualize_output_neuron(model, 385, alpha=0.25) # Indian elephant, Elephas maximus